<div>
<img src="header5.png" width="1000" float='left', style="display:block">
</div>
<a id='tartalomjegyzek'></a>

# Üzleti Intelligencia - Adatelemzési Feladat

**Radó László, MS1PB5. 2020 Április**
<br>
<hr>
<hr>

## 1) Adatok előkészítése


### Tartalomjegyzék:
<blockquote>
<ul>
<li><a href="#bevezeto">Bevezető - Probléma ismertetése</a></li>
<li><a href="#hiba_feltaras">Adathibak Feltárása</a></li>
<li><a href="#hiba_javitas">Megfigyelt problémák, és számítandó új változók listája</a></li>
<li><a href="#tovabbi">További módosítások (későbbi elemzés folyamán szükségessé vált átalakítások)</a></li>
<li><a href="#export">Exportálás</a></li>
</ul>
</blockquote>


<a id='bevezeto'></a>
## Bevezető
<a href="#tartalomjegyzek">Vissza a tartalomjegyzékhez</a>

#### Kiindulási probléma:
- Brazilia Vitória városának egészségügyi létesítményeiben komoly problémát jelent, hogy a pácienjek jelentős hányada nem jelenik meg az előre egyeztetett vizsgálati, vagy kontroll időpontokon.<br>
<br>
- Egy az **[Egyesült Államokban végzett kutatás](https://www.scisolutions.com/uploads/news/Missed-Appts-Cost-HMT-Article-042617.pdf)** szerint a páciensek 30%-a nem jelenik meg az orvosi időpontokon, ami évente 150 milliárd dollár többletterhet ró az egészségügyi szektorra.
- Ezen költségek egy része az adminisztratív többletből, másik része pedig a kihasználatlanul maradt kapacitásokból származik.<br>
<br>
- Brazíliára vonatkozóan hasonló tanulmány nem áll rendelkezésre, viszont ebből a példából látható, hogy a probléma jelentős, így annak mérséklésével, és az egészségügyi ellátórendszer kihasználtságának optimalizálásával komoly gazdasági hatékonyságnövelés lenne elérhető.

<hr>

#### Az elemzési feladat:
 - A rendelkezésre álló adatok alapján azonosítsuk a páciensek azon csoportját, illetve csoportjait, amelyeknél nagyobb arányban fordul elő, hogy nem jelennek meg az időpontokon, illetve tárjuk fel ezen csoportok jellemzőit.
 - Az elemzés végső célja egy olyan gépi tanulás alapú autómatizmus létrehozása, mely a páciens demográfiai illetve kórtörténeti jellemzői alapján megbízható becslést ad arra vonatkozóan, hogy számíthatunk e rá, hogy nem jelenik meg a vizsgálati időponton.
<br>
<br>
 - Ez lehetővé teszi az intézmény számára, hogy az időpontot megelőző napokban az autómatizmus által azonosított pácienst felkeresse és emlékeztesse, illetve meggyőződjön róla, hogy a páciens tervez e megjelenni az időponton. Amennyiben nem, úgy az időpont kiadható más páciens számára.<br>

<hr>

#### A rendelkezésre álló adatállomány:
https://www.kaggle.com/joniarroba/noshowappointments
- Az adatállomány 110.527 orvosi időpontfoglalásról tartalmaz információkat.
- Az adatállomány az alábbi (14) változót tartalmazza:
 - **AppointmentID**: Az időpontfoglalások egyedi azonosítója.
 - **Patientid**: Páciens egyedi azonosítója.
 - **ScheduledDay**: A regisztráció napja.
 - **AppointmentDay**: A legfoglalt időpont (YYYY-MM-DD-HH-MM formátumban)
 - **Gender**: A páciens neme.
 - **Age**: A Páciens életkora.
 - **Neighborhood**: A városrész megnevezése, ahol az egészségügyi intézmény található.
 - **Scholarship (Osztondij)**: Részt vesz e a páciens **['Bolsa Família'](https://en.wikipedia.org/wiki/Bolsa_Fam%C3%ADlia)** nevű állami jóléti programban.
 - **Hipertension (Magas_vernyomas)**: Van e a páciensnek magas vérnyomása (1 igen, 0 nem).
 - **Diabetes (Cukorbetegseg)**: Van e a páciensnek cukorbetegsége (1 igen, 0 nem).
 - **Alcoholism (Alkoholizmus)**: Szenved e a páciens alkoholbetegségben, alkoholizmusban (1 igen, 0 nem).
 - **Handcap (Fogyatekossag)**: A változó értéke azt mutatja hány különböző fogyatékosságban szenved a páciens. A fogyatékosságok fajtáiról nincs információ.
 - **SMS_received (SMS_ertesito)**: Kapott e SMS emlékeztetőt a páciens az időpontot megelőzően. (1 igen, 0 nem).
 - **No-show**: Megjelent e a páciens az időponton ('Yes' megjelent, 'No' nem jelent meg).

_* Zárójelben láthatók a változók magyarítás utáni nevei_

<a id='hiba_feltaras'></a>
## Adathalmaz hibáinak feltárása
<a href="#tartalomjegyzek">Vissza a tartalomjegyzékhez</a>
- Az adathalmaz attribútumainak megismerése
- Adathibák, szükséges korrekciók felmérése

In [1]:
# Adatok és szükséges könyvtárak beolvasása

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import matplotlib.patches as mpatches
%matplotlib inline

no_show =  pd.read_csv('KaggleV2-May-2016.csv', 
                       infer_datetime_format=True, 
                       parse_dates=['ScheduledDay', 'AppointmentDay'])
no_show.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29 18:38:08,2016-04-29,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29 16:08:27,2016-04-29,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29 16:19:04,2016-04-29,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29 17:29:31,2016-04-29,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29 16:07:23,2016-04-29,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [2]:
# Az adathalmas dimenzióinak ellenőrzése.
no_show.shape

(110527, 14)

In [3]:
# Adattípusok és hiányzó értékek számának ellenőrzése.
no_show.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   PatientId       110527 non-null  float64       
 1   AppointmentID   110527 non-null  int64         
 2   Gender          110527 non-null  object        
 3   ScheduledDay    110527 non-null  datetime64[ns]
 4   AppointmentDay  110527 non-null  datetime64[ns]
 5   Age             110527 non-null  int64         
 6   Neighbourhood   110527 non-null  object        
 7   Scholarship     110527 non-null  int64         
 8   Hipertension    110527 non-null  int64         
 9   Diabetes        110527 non-null  int64         
 10  Alcoholism      110527 non-null  int64         
 11  Handcap         110527 non-null  int64         
 12  SMS_received    110527 non-null  int64         
 13  No-show         110527 non-null  object        
dtypes: datetime64[ns](2), float64(1), in

In [4]:
# Változónkénti egyedi értékek számának mefgigyelése.
no_show.nunique()

PatientId          62299
AppointmentID     110527
Gender                 2
ScheduledDay      103549
AppointmentDay        27
Age                  104
Neighbourhood         81
Scholarship            2
Hipertension           2
Diabetes               2
Alcoholism             2
Handcap                5
SMS_received           2
No-show                2
dtype: int64

In [5]:
# Numerikus változók értékei reális tartományban vannak e.
no_show.describe()

,PatientId,AppointmentID,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
count,1.105270e+05,1.105270e+05,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000
mean,1.474963e+14,5.675305e+06,37.088874,0.098266,0.197246,0.071865,0.030400,0.022248,0.321026
std,2.560949e+14,7.129575e+04,23.110205,0.297675,0.397921,0.258265,0.171686,0.161543,0.466873
min,3.921784e+04,5.030230e+06,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.172614e+12,5.640286e+06,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.173184e+13,5.680573e+06,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.439172e+13,5.725524e+06,55.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,9.999816e+14,5.790484e+06,115.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000


In [6]:
no_show.Handcap.value_counts()

0    108286
1      2042
2       183
3        13
4         3
Name: Handcap, dtype: int64



<a id='hiba_javitas'></a>
<a href="#tartalomjegyzek">Vissza a tartalomjegyzékhez</a>

## Megfigyelt problémák, és számítandó új változók:

<br>
<a href="#feladat-1">1) Hibás adattípusok</a>

 * a) PatientId változó adattípusa float64. Helyes: int
 * b) No-show változó adattípusa Object, és értékei szövegesek (Yes/No). Helyes: int, dummy 0/1
 * c) Gender változó adattípusa Object (String). Helyes: int, dummy 0/1
        
<a href="#feladat-2">2) Krónikus betegségek oszlopainak rendezése</a>
 
 * a) Hipertension, Diabetes, Alcoholism, Handcap változók magyarítása (vizuális adatalemzéshez)
 * b) Handicap változó 0-4 közötti értéket tud felvenni. Készítsünk hozzá egy 0-1 értékű oszlopot is.
 * *c) Új változó létrehozása betegségek száma alapján:

<a href="#feladat-3">3) Hozzunk létre dummy változókat a 'Neighbourhood' oszlop értékeiből.</a>

<a href="#feladat-4">4) ScheduleDay változó értékeinek külön változókba rendezése</a>    
 * *a) Hozzunk létre új változót, ami a foglalt időpont óra értékét tartalmazza.
 * *b) Hozzunk létre új változót, ami a rögzíti, hogy a foglalt időpont a hét melyik napjára esik
 * c) Távolítsuk el az óra-perc értékeket az eredeti 'ScheduleDay' változóból

<a href="#feladat-5">5) Hozzunk létre új változót, ami a regisztráció és a vizsgálat időpontja között eltelt napok számát rögzíti.</a>
   
 * *a) Számítsuk ki az eltelt napok számát
 * b) Ellenőrizzük, hogy van e az a) pontban kiszámolt értékek között negatív

<a href="#feladat-6">6) Számítsuk ki a vizsgálat időpontját megelőző korábbi:</a>

 * *a) Időpontfoglalások számát
 * *b) Időpontfoglalásokon történő megjelenések számát
 * *c) Időpontfoglalásokon történő meg NEM jelenések számát
 * *d) Megjelenések aránya az időpontfoglalások számához képest
 * *e) Meg NEM jelenések aránya az időpontfoglalások számához képest
 * *f) Meg NEM jelenések aránya a megjenelések számához képest
 
<a href="#feladat-7">7) Életkor (Age) változó negatív értékeket is tartalmaz.</a>

<a href="#feladat-8">8) További változók magyarítása, illetve helyesírási hibák javítása.</a>

<a id='feladat-1'></a>

### Feladat 1)
**Hibás adattípusok korrigálása** <br>
<a href="#hiba_javitas">Vissza a listához</a>
#### a) _'PatientId'_ változó integer típusra változtatása

In [7]:
no_show.PatientId = no_show.PatientId.astype('int64')

# módosítások ellenőrzése
no_show.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   PatientId       110527 non-null  int64         
 1   AppointmentID   110527 non-null  int64         
 2   Gender          110527 non-null  object        
 3   ScheduledDay    110527 non-null  datetime64[ns]
 4   AppointmentDay  110527 non-null  datetime64[ns]
 5   Age             110527 non-null  int64         
 6   Neighbourhood   110527 non-null  object        
 7   Scholarship     110527 non-null  int64         
 8   Hipertension    110527 non-null  int64         
 9   Diabetes        110527 non-null  int64         
 10  Alcoholism      110527 non-null  int64         
 11  Handcap         110527 non-null  int64         
 12  SMS_received    110527 non-null  int64         
 13  No-show         110527 non-null  object        
dtypes: datetime64[ns](2), int64(9), obje

#### b) Hozunk létre egy új változót, _'No-show_dummy'_ néven a következők szerint:
 - 0, ha megjelent (eredeti érték: No)
 - 1, ha nem jelent meg (eredeti érték: Yes)
 
 <p>
    
A vizuális adatelemzés megkönnyítéséhez az eredeti 'No-show' változó értékeit változtassuk meg a következők szerint:
 - 'Megjelent' (eredeti érték: No)
 - 'Nem jelent meg' (eredeti érték: Yes)

In [8]:
# dummy változó létrehozása
no_show['dummy_No-show'] = no_show['No-show'].apply(lambda x: 1 if x == 'No' else 0)

# eredeti változó magyarítása
no_show['No-show'] = no_show['No-show'].apply(lambda x: 'Megjelent' if x == 'No' else 'Nem jelent meg')


# módosítások ellenőrzése
# a két 'No-show' oszlop index értékeinek összehasonlítása
# megfelelő értékek esetén 'True' választ kapunk

count_no_fixed = no_show[no_show['No-show']=='Megjelent'].index
count_no_dummy = no_show[no_show['dummy_No-show']==1].index
print(list(count_no_dummy) == list(count_no_fixed))

True


#### c) Hozunk létre új változót, _'Gender_dummy'_ néven a következők szerint:
 - 0, ahol az eredeti érték: M
 - 1, ahol az eredeti érték: F
 
<p>
    
A vizuális adatelemzés megkönnyítése érdekében változtassuk meg a 'Gender' változó értékeit az alábbiak szerint
 - 'Férfi', ahol az eredeti érték: M
 - 'Nő', ahol az eredeti érték: F

In [9]:
# dummy létrehozása
no_show['dummy_Gender'] = no_show['Gender'].apply(lambda x: 1 if x == 'F' else 0)

# eredeti változó magyarítása
no_show['Gender'] = no_show['Gender'].apply(lambda x: 'Nő' if x == 'F' else 'Férfi')


# módosítások ellenőrzése
# a két Gender oszlop index értékeinek összehasonlítása
# megfelelő értékek esetén 'True' választ kapunk
count_F_fixed = no_show[no_show['Gender']=='Nő'].index
count_F_dummy = no_show[no_show['dummy_Gender']==1].index
print(list(count_F_fixed) == list(count_F_dummy))

True


<a id='feladat-2'></a>
### Feladat 2):
**Betegségek oszlopainak rendezése**<br>
<a href="#hiba_javitas">Vissza a listához</a>

#### a) Hipertension, Diabetes, Alcoholizm és Handcap változók magyarítása

In [10]:
no_show.rename(columns={'Hipertension' : 'Magas_vernyomas',
                        'Diabetes' : 'Cukorbeteseg',
                        'Alcoholism' : 'Alkoholizmus',
                        'Handcap' : 'Fogyatekossag' ,
                        }, inplace=True)

# módosítások ellenőrzése
no_show.head(0)

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Magas_vernyomas,Cukorbeteseg,Alkoholizmus,Fogyatekossag,SMS_received,No-show,dummy_No-show,dummy_Gender


#### b) Handicap változó 0-4 közötti értéket tud felvenni. Készítsünk hozzá egy 0-1 értékű oszlopot is az alábbiak szerint:
   * Ha Handicap értéke > 0, akkor az új változó értéke = 1
   * Egyéb esetben a változó értéke 0

In [11]:
no_show['Fogyatekossag_binary'] = [1 if x > 0 else 0 for x in no_show['Fogyatekossag']]

# módosítások ellenőrzése:
no_show[['Fogyatekossag', 'Fogyatekossag_binary']][no_show['Fogyatekossag'] == 4]

,Fogyatekossag,Fogyatekossag_binary
91820,4,1
98538,4,1
104268,4,1


#### c) Új változó létrehozása betegségek száma alapján:

In [12]:
no_show['betegsegek_szama'] = no_show[
    ['Magas_vernyomas', 'Cukorbeteseg', 'Alkoholizmus', 'Fogyatekossag_binary']].sum(axis=1)

<a id='feladat-3'></a>
### Feladat 3):
**Hozzunk létre dummy változókat a 'Neighbourhood' változó értékeiből.**<br>
<a href="#hiba_javitas">Vissza a listához</a>

In [13]:
no_show = pd.concat([no_show, pd.get_dummies(no_show['Neighbourhood'], prefix='dummy')], axis=1)
no_show.columns = no_show.columns.str.replace(' ', '_')

# megoldások ellenőrzése
no_show[['Neighbourhood', 'dummy_JARDIM_DA_PENHA', 'dummy_MATA_DA_PRAIA', 'dummy_PONTAL_DE_CAMBURI']].head()

,Neighbourhood,dummy_JARDIM_DA_PENHA,dummy_MATA_DA_PRAIA,dummy_PONTAL_DE_CAMBURI
0,JARDIM DA PENHA,1,0,0
1,JARDIM DA PENHA,1,0,0
2,MATA DA PRAIA,0,1,0
3,PONTAL DE CAMBURI,0,0,1
4,JARDIM DA PENHA,1,0,0


In [14]:
list(no_show.columns)

['PatientId',
 'AppointmentID',
 'Gender',
 'ScheduledDay',
 'AppointmentDay',
 'Age',
 'Neighbourhood',
 'Scholarship',
 'Magas_vernyomas',
 'Cukorbeteseg',
 'Alkoholizmus',
 'Fogyatekossag',
 'SMS_received',
 'No-show',
 'dummy_No-show',
 'dummy_Gender',
 'Fogyatekossag_binary',
 'betegsegek_szama',
 'dummy_AEROPORTO',
 'dummy_ANDORINHAS',
 'dummy_ANTÔNIO_HONÓRIO',
 'dummy_ARIOVALDO_FAVALESSA',
 'dummy_BARRO_VERMELHO',
 'dummy_BELA_VISTA',
 'dummy_BENTO_FERREIRA',
 'dummy_BOA_VISTA',
 'dummy_BONFIM',
 'dummy_CARATOÍRA',
 'dummy_CENTRO',
 'dummy_COMDUSA',
 'dummy_CONQUISTA',
 'dummy_CONSOLAÇÃO',
 'dummy_CRUZAMENTO',
 'dummy_DA_PENHA',
 'dummy_DE_LOURDES',
 'dummy_DO_CABRAL',
 'dummy_DO_MOSCOSO',
 'dummy_DO_QUADRO',
 'dummy_ENSEADA_DO_SUÁ',
 'dummy_ESTRELINHA',
 'dummy_FONTE_GRANDE',
 'dummy_FORTE_SÃO_JOÃO',
 'dummy_FRADINHOS',
 'dummy_GOIABEIRAS',
 'dummy_GRANDE_VITÓRIA',
 'dummy_GURIGICA',
 'dummy_HORTO',
 'dummy_ILHA_DAS_CAIEIRAS',
 'dummy_ILHA_DE_SANTA_MARIA',
 'dummy_ILHA_DO_BOI',

<a id='feladat-4'></a>
### Feladat 4):
**ScheduleDay változó értékeinek külön változókba rendezése**<br>
<a href="#hiba_javitas">Vissza a listához</a>

#### a) Vizsgálati időpont: óra érték rögzítése külön változóba

In [15]:
no_show['ScheduledHour'] = no_show['ScheduledDay'].dt.hour
no_show[['ScheduledDay','ScheduledHour']].head()

,ScheduledDay,ScheduledHour
0,2016-04-29 18:38:08,18
1,2016-04-29 16:08:27,16
2,2016-04-29 16:19:04,16
3,2016-04-29 17:29:31,17
4,2016-04-29 16:07:23,16


#### b) Vizsgálati időpont: hét napjának rögzítése külön változóba

In [16]:
no_show['DayofWeek'] = no_show['ScheduledDay'].dt.dayofweek
no_show[['ScheduledDay','DayofWeek']].sample(5)

,ScheduledDay,DayofWeek
92565,2016-05-18 14:48:04,2
84765,2016-06-03 11:50:37,4
8354,2016-04-25 14:32:41,0
20433,2016-05-05 15:30:55,3
82718,2016-05-02 08:46:16,0


#### c) Vizsgálati időpont: óra-perc érték eltávolítása az eredeti változóból

In [17]:
no_show['ScheduledDay'] = no_show['ScheduledDay'].apply(lambda x: x.strftime('%m-%d-%Y'))
no_show['ScheduledDay'] = pd.to_datetime(no_show['ScheduledDay'])
no_show['ScheduledDay'].head()

0   2016-04-29
1   2016-04-29
2   2016-04-29
3   2016-04-29
4   2016-04-29
Name: ScheduledDay, dtype: datetime64[ns]

<a id='feladat-5'></a>
### Feladat 5):
**Hozzunk létre új változót, ami a regisztráció és a vizsgálat időpontja között eltelt napok számát rögzíti.**<br>
<a href="#hiba_javitas">Vissza a listához</a>

#### a) Számítsuk ki az eltelt napok számát

In [18]:
no_show['Num_days_to_appointment'] =  no_show['AppointmentDay'] - no_show['ScheduledDay']
no_show['Num_days_to_appointment'] = no_show['Num_days_to_appointment'].dt.days

# módosítások ellenőrzése
no_show.sample(3)

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Magas_vernyomas,Cukorbeteseg,...,dummy_SÃO_BENEDITO,dummy_SÃO_CRISTÓVÃO,dummy_SÃO_JOSÉ,dummy_SÃO_PEDRO,dummy_TABUAZEIRO,dummy_UNIVERSITÁRIO,dummy_VILA_RUBIM,ScheduledHour,DayofWeek,Num_days_to_appointment
101873,667635397556478,5765682,Nő,2016-06-02,2016-06-02,5,RESISTÊNCIA,0,0,0,...,0,0,0,0,0,0,0,13,3,0
3485,291145987362399,5604918,Nő,2016-04-20,2016-05-11,12,RESISTÊNCIA,0,0,0,...,0,0,0,0,0,0,0,8,2,21
78354,25557852993893,5673714,Nő,2016-05-09,2016-05-11,36,JARDIM CAMBURI,0,0,0,...,0,0,0,0,0,0,0,9,0,2


#### b) Ellenőrizzük, hogy van e az a) pontban kiszámolt értékek között negatív

In [19]:
no_show[no_show['Num_days_to_appointment'] < 0].head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Magas_vernyomas,Cukorbeteseg,...,dummy_SÃO_BENEDITO,dummy_SÃO_CRISTÓVÃO,dummy_SÃO_JOSÉ,dummy_SÃO_PEDRO,dummy_TABUAZEIRO,dummy_UNIVERSITÁRIO,dummy_VILA_RUBIM,ScheduledHour,DayofWeek,Num_days_to_appointment
27033,7839272661752,5679978,Férfi,2016-05-10,2016-05-09,38,RESISTÊNCIA,0,0,0,...,0,0,0,0,0,0,0,10,1,-1
55226,7896293967868,5715660,Nő,2016-05-18,2016-05-17,19,SANTO ANTÔNIO,0,0,0,...,0,0,0,0,0,0,0,14,2,-1
64175,24252258389979,5664962,Nő,2016-05-05,2016-05-04,22,CONSOLAÇÃO,0,0,0,...,0,0,0,0,0,0,0,13,3,-1
71533,998231581612122,5686628,Nő,2016-05-11,2016-05-05,81,SANTO ANTÔNIO,0,0,0,...,0,0,0,0,0,0,0,13,2,-6
72362,3787481966821,5655637,Férfi,2016-05-04,2016-05-03,7,TABUAZEIRO,0,0,0,...,0,0,0,0,1,0,0,6,2,-1


In [20]:
# Mivel a negatív különbséggel rendelkező rekordok száma alacsony (5), 
# és nem tudjuk miért negatív az érték (adatrögzítési hiba, vagy valóban utólagosan lett rögzítve az időpont)
# ezért ezeket a rekordokat eltávolítjuk.

no_show.drop(index=no_show[no_show['Num_days_to_appointment'] < 0].index, inplace=True)

In [21]:
no_show[no_show['Num_days_to_appointment'] < 0]

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Magas_vernyomas,Cukorbeteseg,...,dummy_SÃO_BENEDITO,dummy_SÃO_CRISTÓVÃO,dummy_SÃO_JOSÉ,dummy_SÃO_PEDRO,dummy_TABUAZEIRO,dummy_UNIVERSITÁRIO,dummy_VILA_RUBIM,ScheduledHour,DayofWeek,Num_days_to_appointment


<a id='feladat-6'></a>
### Feladat 6) 
**Vizsgálat időpontját megelőző korábbi események összegzése**<br>
<a href="#hiba_javitas">Vissza a listához</a>

#### a) Időpontfoglalások száma

In [22]:
# Páciens adott időpontot megelőző korábbi látogatásainak száma
no_show['number_of_visits'] = 0
for i in no_show.index:
    count = no_show[(no_show['PatientId'] == no_show.loc[i, 'PatientId']) & 
                    (no_show['AppointmentDay'] < no_show.loc[i, 'AppointmentDay'])].shape[0]
    no_show.at[i, 'number_of_visits'] = count
    
# Bináris, gyakori látogató oszlop létrehozása 
# 1, ha már volt vizsgálaton korábban, 0 ha még nem volt időpontja
no_show['frequent_visitor'] = [0 if no_show.at[i, 'number_of_visits'] == 0 else 1 for i in no_show.index]

#### b) Időpontfoglalásokon történő megjelenések számát

In [23]:
# Páciens adott időpontot megelőző késéseinek száma
no_show['number_of_shows'] = 0
for i in no_show.index:
    count = no_show[(no_show['PatientId'] == no_show.loc[i, 'PatientId']) & 
                    (no_show['AppointmentDay'] < no_show.loc[i, 'AppointmentDay']) & 
                    (no_show['No-show'] == 'Megjelent')].shape[0]
    no_show.at[i, 'number_of_shows'] = count
    
# Bináris, késések száma
# az érték 1, ha már késett korábban, 0 ha még soha nem késett
no_show['frequent_show'] = [0 if no_show.at[i, 'number_of_shows'] == 0 else 1 for i in no_show.index]

#### c) Időpontfoglalásokon történő meg NEM jelenések számát

In [24]:
# Páciens adott időpontot megelőző késéseinek száma
no_show['number_of_no_shows'] = 0
for i in no_show.index:
    count = no_show[(no_show['PatientId'] == no_show.loc[i, 'PatientId']) & 
                    (no_show['AppointmentDay'] < no_show.loc[i, 'AppointmentDay']) & 
                    (no_show['No-show'] == 'Nem jelent meg')].shape[0]
    no_show.at[i, 'number_of_no_shows'] = count
    
# Bináris, késések száma
# az érték 1, ha már késett korábban, 0 ha még soha nem késett
no_show['frequent_no_show'] = [0 if no_show.at[i, 'number_of_no_shows'] == 0 else 1 for i in no_show.index]

#### d) Megjelenések aránya az időpontfoglalások számához képest

In [25]:
no_show['prop_show'] = no_show['number_of_shows'] / no_show['number_of_visits']
no_show['prop_show'].fillna(1, inplace=True)

#### e) Meg NEM jelenések aránya az időpontfoglalások számához képest

In [26]:
no_show['prop_no_show'] = no_show['number_of_no_shows'] / no_show['number_of_visits']
no_show['prop_no_show'].fillna(1, inplace=True)


C:\ProgramData\Miniconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


#### f) Meg NEM jelenések aránya a megjenelések számához képest

In [27]:
no_show['prop_show_noshow'] = no_show['number_of_no_shows'] / no_show['number_of_visits']
no_show['prop_show_noshow'].fillna(1, inplace=True)

<a id='feladat-7'></a>
### Feladat 7):
**Életkor (Age) változó negatív értékeket is tartalmaz. Ezeket a rekordokat töröljük.**<br>
<a href="#hiba_javitas">Vissza a listához</a>

In [28]:
# Negatív értéket tartalmazó rekordok szűrése
no_show[no_show['Age']<0]

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Magas_vernyomas,Cukorbeteseg,...,Num_days_to_appointment,number_of_visits,frequent_visitor,number_of_shows,frequent_show,number_of_no_shows,frequent_no_show,prop_show,prop_no_show,prop_show_noshow
99832,465943158731293,5775010,Nő,2016-06-06,2016-06-06,-1,ROMÃO,0,0,0,...,0,0,0,0,0,0,0,1.0,0.0,1.0


In [29]:
# Negatív értéket tartalmazó rekordok törlése
no_show.drop(index=no_show[no_show['Age']<0].index, inplace=True)

# módosítások ellenőrzése
no_show[no_show['Age']<0]

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Magas_vernyomas,Cukorbeteseg,...,Num_days_to_appointment,number_of_visits,frequent_visitor,number_of_shows,frequent_show,number_of_no_shows,frequent_no_show,prop_show,prop_no_show,prop_show_noshow


<a id='feladat-8'></a>
### Feladat 8):
**További változók magyarítása, illetve helyesírási hibák javítása.**<br>
<a href="#hiba_javitas">Vissza a listához</a>

In [30]:
no_show.rename(columns={'SMS_received' : 'SMS_ertesito',
                        'Scholarship' : 'Osztondij',
                        'Neighbourhood' : 'Neighborhood'}, inplace=True)

<a id='tovabbi'></a>
# További Átalakítások
<a href="#tartalomjegyzek">Vissza a tartalomjegyzékhez</a>

### Modellezés közben megfigyelt jelenségek kezelése
Az alábbi változók estében, a mintaméret jelentősen lecsökken egy bizonyos érték fölött<br>
**Például:** 
* Az életkor (Age) változó esetében 80 év fölött csökken a mintaméret. A kis minta miatt, nehéz megfigyelni, hogy a 80 évet meghaladó életkorok milyen kapcsolatban vannak a vizsgált y változónkkal (No-show).
* Hasonló jelenség figyelhető meg az alábbi változók esetében:
 * betegsegek_szama
 * number_of_no_shows
 * number_of_shows
 * number_of_visits
 * number_of_days_to_appointment_capped
* Az érintett változók ezen tartományba tartozó értékeit külön kategóriába rendezzük. Ehhez egy új változót hozunk létre (hogy az eredeti értékek is megmaradjanak)
 * Például: az életkor (Age) változó esetében a 80 év feletti páciensek mind 80 évesként jelennek meg.

In [31]:
no_show['Age_capped_80'] = [
    80 if i >= 80 else i for i in no_show['Age']]

no_show['betegsegek_szama_capped'] = [
    2 if i >= 2 else i for i in no_show['betegsegek_szama']]
    
no_show['number_of_no_shows_capped'] = [
    6 if i > 6 else i for i in no_show['number_of_no_shows']]

no_show['number_of_shows_capped'] = [
    15 if i >= 15 else i for i in no_show['number_of_shows']]
    
no_show['number_of_visits_capped'] = [
    2 if i > 2 else i for i in no_show['number_of_visits']]
    
no_show['number_of_visits_capped_mult'] = [
    2 if (i >= 2) & (i < 10) else 3 if i >= 10 else i for i in no_show['number_of_visits']]
    
no_show['Num_days_to_appointment_capped'] = [
    12 if i >= 12 else i for i in no_show['Num_days_to_appointment']]


#### További dummy változók létrehozása
* Az előjegyzés időpontja (ScheduledHour)

In [33]:
# dummy változók képzése
no_show = pd.concat([no_show, pd.get_dummies(no_show['ScheduledHour'], prefix='dummy_hour_')], axis=1)
no_show.columns = no_show.columns.str.replace(' ', '_')

* Életkor Kvartilisek

In [35]:
# Kvartilisek létrehozása
no_show['Age_Quantile'] = pd.qcut(no_show['Age'], 4)

# Kvartilisekből dummy változók képzése
pd.concat([no_show, pd.get_dummies(no_show['Age_Quantile'], prefix='dummy_age_')], axis=1)
no_show.columns = no_show.columns.str.replace(' ', '_')

#### Négyzetes transzformáció
* Korábbi megjelenések száma (number_of_shows)
* Korábbi előjegyzések száma (number_of_visits)

In [36]:
no_show['number_of_shows_capped_sq'] = [
    np.power(x,2) for x in no_show['number_of_shows_capped']]

no_show['number_of_shows_sq'] = [
    np.power(x,2) for x in no_show['number_of_shows']]

no_show['number_of_visits_capped_sq'] = [
    np.power(x,2) for x in no_show['number_of_visits_capped']]

no_show['number_of_visits_sq'] = [
    np.power(x,2) for x in no_show['number_of_visits']]

no_show['number_of_visits_capped_mult_sq'] = [
    np.power(x,2) for x in no_show['number_of_visits_capped_mult']]

#### Log transzformáció

In [37]:
no_show['prop_show_log'] = [
    np.power(x,2) for x in no_show['prop_show']]

no_show['prop_no_show_log'] = [
    np.power(x,2) for x in no_show['prop_no_show']]

no_show['prop_show_noshow_log'] = [
    np.power(x,2) for x in no_show['prop_show_noshow']]

<a id='export'></a>
## Exportálás
<a href="#tartalomjegyzek">Vissza a tartalomjegyzékhez</a>

Tisztított adatforrás exportálása csv file-ba.

In [38]:
no_show.to_csv('KaggleV2-May-2016_clean.csv')